In [ ]:
#ライブラリの自動リロード
%load_ext autoreload
%autoreload 2

import openai
from settings.key import GPT_API_KEY
openai.api_key = GPT_API_KEY



In [ ]:
from LLMSearch.AnswerBot import AnswerBot,parse_answer
from settings.key import GPT_API_KEY, DEEPL_API_KEY
from LLMSearch.GPTQuery import GPTQuery
from LLMSearch.BM25DB import BM25DB
from LLMSearch.SQLTextDB import SQLTextDB
from LLMSearch.Embedding.SBERTFineTuneEmbedding import SBERTFineTuneEmbedding
from LLMSearch.VectorSearcher import VectorSearcher

embedder=SBERTFineTuneEmbedding()
searcher=VectorSearcher(embedder)


#bm=BM25DB(initiate=True)
#searcher=SQLTextDB()

# initiate bot module
bot1 = AnswerBot(query_module=GPTQuery(GPT_API_KEY),
                searcher=searcher,
                DEEPL_API_KEY=DEEPL_API_KEY,
                )

bot2=AnswerBot(query_module=GPTQuery(GPT_API_KEY),
                searcher=SQLTextDB(), 
                DEEPL_API_KEY=DEEPL_API_KEY,
                )

bot3=AnswerBot(query_module=GPTQuery(GPT_API_KEY),
                searcher=BM25DB(),
                DEEPL_API_KEY=DEEPL_API_KEY,   
                )

In [ ]:
bot1.index_documents(initiate=False)
bot2.index_documents(initiate=False)
bot3.index_documents(initiate=False)

In [30]:
query="先生の業績は?"
query="先生の趣味は?"
query="先生は何故自殺をしたのか?"
#query="藤田正博先生の業績は?"
#query="藤田正博先生の業績は?"
query="イオン伝導度を上げるためには?"

related_documents=[]
for bot in [bot1,bot2,bot3]:
    related_documents+=bot.search_related_documents(query, k=10)

ans={}
ans["answer"]=""
ans["context"]=related_documents
#str_ans=parse_answer(ans)
#print(str_ans)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
import json
import numpy as np

#related documentsから､重複を除く
#related_documents=list(set(related_documents))
#list_of_json = [json.dumps(d, sort_keys=True,cls=MyEncoder) for d in related_documents]

#related_documents=[{"path":d["path"],"text":d["text"],"sim":float(d["sim"])} for d in related_documents]
related_documents=[{"path":d["path"],"text":d["text"],"sim":0} for d in related_documents]
list_of_json = [json.dumps(d, sort_keys=True) for d in related_documents]
related_documents= list(map(json.loads, set(list_of_json)))
print(len(related_documents))
related_documents

27


[{'path': 'UserData/original/text/kobunshi_ronbunshu/koron_74_6_74_2017-0034.txt',
  'sim': 0,
  'text': 'https.//www.jstage.jst.go.jp/article/koron/74/6/74_2017-0034/_pdf/-char/ja.高分子論文集〔原著論文〕五員環カーボナート基をもつポリシロキサンの固体高分子電解質としての応用松本幸三*1・堤大介*1・桑島信*1・遠藤剛*2（受付2017年4月28日・審査終了2017年7月6日）（早期公開2017年8月18日・DOI.10.1295 /koron.2017 -0034）.要旨五員環環状カーボナート基をもつポリシロキサン (polyMSOC )にリチウムトリフルオロメタンスルホナート (LiOTf )，またはリチウムビス (トリフルオロメタンスルホニル )イミド (LiTFSI )を添加し，イオン伝導度を測定した.LiOTfを添加した場合は，カーボナート基に対して 2.0等量の塩添加量でイオン伝導度が最大となり，30°Cで3.6×10－8S/cmを示した.一方，LiTFSIを添加した場合は，塩添加量の増加に従いイオン伝導度が上昇し，カーボナート基に対して 4.0等量の塩添加量時には，30°Cで7.7×10－5S/cmの高いイオン伝導度を示した..1緒言リチウムイオンバッテリーは， サイクル寿命が長く，高エネルギー密度，かつ高出力であることから ，ノートパソコンやスマートフォン ，電気自動車(EV )やプラグインハイブリッドカー (PHEV)などさまざまな機器の電源として広く利用されている1),2).しかしながら ，通常は可燃性有機電解液を使用することから， 電解液の液漏れや発火などの危険性が危惧されている.この問題を解決するために ，固体高分子電解質を使用する試みが多くなされている3).これまでは ，おもにポリエチレンオキサイド (PEO )系のポリマーが検討されてきたが4)，イオン伝導性が不十分であることなどから ，固体高分子電解質の普及は遅れているのが現状である..これに対して，近年カーボナート系のポリマーが注目され始めている.たとえば ，富永らは，ポリエチレンカーボナートがリチウムビス 

In [32]:
#related_documentを5こずつに分割
related_documents_list=[related_documents[i:i+5] for i in range(0,len(related_documents),5)]
#related_documents_list[1]

In [33]:
for related_documents in related_documents_list:
    response=bot1.ask(query,context_list= related_documents,stream=True)

    ans=response["answer"]
    for chunk in ans:
        next = chunk['choices'][0]['delta'].get('content', '')
        print(next, end='')

    print("\n")

リチウムイオン伝導体において、イオン伝導度を上げるためには、カーボナート基に対して十分な塩の添加が必要である。具体的には、リチウムトリフルオロメタンスルホナートやリチウムビス(トリフルオロメタンスルホニル)イミドなど、適切な量の塩を添加することで、より高いイオン伝導度を得ることができる。その際、塩の添加量や種類によってイオン伝導度の変化が異なるため、最適な条件を見つける必要がある。

イオン伝導度の向上には、キャリアイオンの増加と同時に、擬似的な架橋構造を防ぐことが重要である。ポリカーボネートは、高塩濃度においても擬似的な架橋構造の影響が少なく、イオンの移動を阻害せずキャリアイオンを増加させることができると期待されている。したがって、ポリカーボネートを使用することが一つの方法である。

イオン伝導度を上げるためには、分子結晶中にイオン伝導パスを構築することが重要である。また、ポリマーの場合は結晶化は避ける必要がある。セラミック電解質では、結晶構造中に連続した空隙構造が存在し、伝導パスを介してリチウムイオンが拡散していることが示されている。さらに、伝導パスの適切な設計により、優れたリチウムイオン伝導性を示すセラミック電解質も合成されている。

高極性分子をマトリックスに用いることで、高濃度の塩を溶解できるため、導電率を容易に上げることが可能である。また、導電性充てん剤をポリウレタンマトリックスに分散させた系もある。さらに、ポリウレタンアイオノマー(PUI)を使用することもできる。

The given context does not contain information answering the question "What should be done to increase the ionic conductivity?"

イオン伝導度を上げるためには、易動度の高いイオンを使用することや、塩をよく溶解する高分子マトリックスを使用すること、Tgが低いアモルファスな高分子を選ぶことなどが挙げられる。また、分子設計や架橋などの手法を用いて、適切な高分子固体電解質を作製することが必要である。

